# **Context, Question, Answer Dataset Creation**

## **Import Libraries**

In [31]:
import os
import json
import fitz
import math
from paddleocr import PaddleOCR
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from tqdm.auto import tqdm
import paddle

pd.set_option('display.max_colwidth', None)

In [32]:
import paddle
print("CUDA available:", paddle.is_compiled_with_cuda())

CUDA available: True


In [33]:
# 1. Load env
load_dotenv()

# 2. Init new OpenAI client (it picks up OPENAI_API_KEY automatically)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 3. Init OCR
ocr = PaddleOCR(use_angle_cls=True, lang="id", show_log=False, use_gpu=True)

# 4. pdf root
pdf_root = "pojk-main-documents-filtered"

[2025/05/01 16:15:27] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/05/01 16:15:28] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2025/05/01 16:15:30] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0


## **Load Metadata**

In [34]:
df_metadata = pd.read_csv('METADATA_USED.csv')
print("Jumlah Dokumen:", len(df_metadata))

Jumlah Dokumen: 206


In [35]:
# 1. Pastikan 'pages' sudah ada di df_metadata
# 2. Hitung nilai maksimum (dibulatkan ke atas jika perlu)
max_page = int(math.ceil(df_metadata['pages'].max()))

# 3. Buat bins dengan lebar 5: 0, 5, 10, ..., sampai melewati max_page
bins = list(range(0, (max_page // 5 + 2) * 5, 5))

# 4. Siapkan label: "0-5", "6-10", "11-15", dst.
labels = []
for i in range(len(bins) - 1):
    start, end = bins[i], bins[i+1]
    if i == 0:
        labels.append(f"{start}-{end}")
    else:
        labels.append(f"{start+1}-{end}")

# 5. Tambahkan kolom 'page_range' dengan interval 5 halaman
df_metadata['page_range'] = pd.cut(
    df_metadata['pages'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

# 6. Hitung frekuensi tiap rentang dan tampilkan sebagai tabel
table = (
    df_metadata['page_range']
      .value_counts(sort=False)
      .reset_index()
      .rename(columns={'index': 'Page Range', 'page_range': 'Count'})
)

print(table)

   Count  count
0    0-5      1
1   6-10     34
2  11-15     46
3  16-20     42
4  21-25     33
5  26-30     23
6  31-35     27
7  36-40      0


In [36]:
# Path folder tempat file PDF disimpan
pdf_root = "pojk-main-documents-filtered"

# Cetak satu path lengkap dokumen OCR
print("📄 Satu Contoh Dokumen OCR:")
fn_ocr = df_metadata[df_metadata['is_scanned'] == True]['new_filename'].iloc[0]
print(os.path.join(pdf_root, fn_ocr))

# Cetak satu path lengkap dokumen Non-OCR
print("\n📄 Satu Contoh Dokumen Non-OCR:")
fn_non_ocr = df_metadata[df_metadata['is_scanned'] == False]['new_filename'].iloc[0]
print(os.path.join(pdf_root, fn_non_ocr))


📄 Satu Contoh Dokumen OCR:
pojk-main-documents-filtered\ojk-peraturan_ojk-18_pojk_03_2014-19112014-pojk_tentang_penerapan_tata_kelola_terintegrasi_bagi_konglomerasi_keuangan.pdf

📄 Satu Contoh Dokumen Non-OCR:
pojk-main-documents-filtered\ojk-peraturan_ojk-6_tahun_2024-02052024-pembiayaan_transaksi_efek_oleh_perusahaan_efek_bagi_nasabah_dan_transaksi_short_selling_oleh_perusahaan_efek.pdf


## **Context, Question, Answer Dataset Generation**

### **Helper Functions**

In [37]:
def extract_text_from_pdf(file_path: str, ocr, threshold: float = 0.95) -> str:
    doc = fitz.open(file_path)
    all_pages_text = []

    for page in doc:
        # 1) Text‐layer
        text = page.get_text("text") or ""

        # 2) OCR pada gambar
        for img in page.get_images(full=True):
            xref = img[0]
            pix  = fitz.Pixmap(doc, xref)
            img_bytes = pix.tobytes()

            # jalankan OCR
            ocr_results = ocr.ocr(img_bytes)
            if not ocr_results:
                continue

            for line in ocr_results:
                if not line:
                    continue
                for word in line:
                    txt, conf = word[1]
                    if conf >= threshold:
                        text += " " + txt

        all_pages_text.append(text.strip())

    return "\n\n".join([p for p in all_pages_text if p])


In [38]:
## **Helper: jumlah QA pairs berdasarkan halaman**
def get_num_pairs(page_count: int) -> int:
    if   page_count <=  5: return 1
    elif page_count <= 10: return 2
    elif page_count <= 15: return 3
    elif page_count <= 20: return 4
    elif page_count <= 25: return 5
    elif page_count <= 30: return 6
    else:                  return 7

In [39]:
def generate_cqa(full_text: str,
                 n_pairs: int,
                 model: str = "gpt-4o-mini-2024-07-18",
                 max_tokens: int = 2048) -> list[dict]:
    prompt = f"""
Anda adalah pakar regulasi keuangan Indonesia.

Teks dokumen:
\"\"\"{full_text}\"\"\"

1) Jika teks **tidak bermakna** (hanya boilerplate seperti "Cukup jelas", header/footer), 
   balas dengan: []

2) Jika **bermakna**, bagi menjadi **{n_pairs}** konteks dengan panjang **sekitar 500–1000 karakter** per konteks,
   lalu untuk tiap konteks buat:
   - context
   - question
   - answer

3) Buat **answer** dengan **panjang sekitar 64 token** (tidak terlalu pendek, tidak lebih dari 70 token), dan **pastikan jawaban berakhir di akhir kalimat** (tidak terpotong).

Keluarkan **hanya** JSON array, misalnya:
[
  {{ "context":"…","question":"…","answer":"…" }},
  …
]
"""
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role":"system", "content":"Anda membuat QA dataset atau [] jika tidak bermakna."},
            {"role":"user",   "content":prompt}
        ],
        temperature=0.7,
        max_tokens=max_tokens
    )
    return json.loads(resp.choices[0].message.content)


### **Extract Text Results**

#### **OCR**

In [40]:
# # 1. Tentukan file PDF yang akan diekstrak
# fn = fn_ocr
# pdf_path = os.path.join(pdf_root, fn)

# # 2. Panggil fungsi
# full_text = extract_text_from_pdf(pdf_path, ocr, threshold=0.8)

# # 3. Tampilkan cuplikan hasil—misal 500 karakter pertama
# print("=== Cuplikan 500 karakter pertama ===")
# print(full_text[:500] + "…\n")

# # 4. (Opsional) Simpan ke file teks agar mudah ditinjau
# with open("extracted_text_ocr.txt", "w", encoding="utf-8") as f:
#     f.write(full_text)

# print(f"Hasil ekstraksi disimpan di 'extracted_text.txt'")

#### **Non-OCR**

In [41]:
# # 1. Tentukan file PDF yang akan diekstrak
# fn = fn_non_ocr
# pdf_path = os.path.join(pdf_root, fn)

# # 2. Panggil fungsi
# full_text = extract_text_from_pdf(pdf_path, ocr, threshold=0.90)

# # 3. Tampilkan cuplikan hasil—misal 500 karakter pertama
# print("=== Cuplikan 500 karakter pertama ===")
# print(full_text[:500] + "…\n")

# # 4. (Opsional) Simpan ke file teks agar mudah ditinjau
# with open("extracted_text_non_ocr.txt", "w", encoding="utf-8") as f:
#     f.write(full_text)

# print(f"Hasil ekstraksi disimpan di 'extracted_text.txt'")

### **Building the Dataset**

In [42]:
# df_metadata = df_metadata.sample(100, random_state=42).reset_index(drop=True)

print("Jumlah Dokumen:", len(df_metadata))
df_metadata.head(3)

Jumlah Dokumen: 206


,title,page_url,sektor,subsektor,jenis_regulasi,nomor_regulasi,tanggal_berlaku,filename,file_url,new_filename,pages,page_range,is_scanned
0,Pembiayaan Transaksi Efek Oleh Perusahaan Efek Bagi Nasabah dan Transaksi Short Selling Oleh Perusahaan Efek,https://www.ojk.go.id/id/regulasi/Pages/Pembiayaan-Transaksi-Efek-Perusahaan-Efek-Bagi-Nasabah-dan-Transaksi-Short-Selling-Oleh-Perusahaan-Efek-POJK-6-Tahun-2024.aspx,Pasar Modal,Perusahaan Efek,Peraturan OJK,6 Tahun 2024,2 Mei 2024,Pembiayaan-Transaksi-Efek-Perusahaan-Efek-Bagi-Nasabah-dan-Transaksi-Short-Selling-Oleh-Perusahaan-Efek-POJK-6-Tahun- -- POJK 6 Tahun 2024 Pembiayaan Transaksi Efek Oleh Perusahaan Efek Bagi Nasabah dan Transaksi Short Selling Oleh Perusahaan Efek.pdf,https://www.ojk.go.id/id/regulasi/Documents/Pages/Pembiayaan-Transaksi-Efek-Perusahaan-Efek-Bagi-Nasabah-dan-Transaksi-Short-Selling-Oleh-Perusahaan-Efek-POJK-6-Tahun-2024/POJK%206%20Tahun%202024%20Pembiayaan%20Transaksi%20Efek%20Oleh%20Perusahaan%20Efek%20Bagi%20Nasabah%20dan%20Transaksi%20Short%20Selling%20Oleh%20Perusahaan%20Efek.pdf,ojk-peraturan_ojk-6_tahun_2024-02052024-pembiayaan_transaksi_efek_oleh_perusahaan_efek_bagi_nasabah_dan_transaksi_short_selling_oleh_perusahaan_efek.pdf,28,26-30,False
1,Laporan Kepemilikan atau Setiap Perubahan Kepemilikan Saham Perusahaan Terbuka dan Aktivitas Menjaminkan Saham Perusahaan Terbuka,https://www.ojk.go.id/id/regulasi/Pages/POJK-4-Tahun-2024-Laporan-Kepemilikan-atau-Setiap-Perubahan-Kepemilikan-Saham-Perusahaan-Terbuka-dan-Aktivitas-Menjaminkan.aspx,Pasar Modal,Peraturan Lainnya,Peraturan OJK,4 Tahun 2024,28 Februari 2024,POJK-4-Tahun-2024-Laporan-Kepemilikan-atau-Setiap-Perubahan-Kepemilikan-Saham-Perusahaan-Terbuka-dan-Aktivitas-Menjaminkan -- POJK 4 Tahun 2024 Laporan Kepemilikan atau Setiap Perubahan Kepemilik.pdf,https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-4-Tahun-2024-Laporan-Kepemilikan-atau-Setiap-Perubahan-Kepemilikan-Saham-Perusahaan-Terbuka-dan-Aktivitas-Menjaminkan/POJK%204%20Tahun%202024%20Laporan%20Kepemilikan%20atau%20Setiap%20Perubahan%20Kepemilik.pdf,ojk-peraturan_ojk-4_tahun_2024-28022024-laporan_kepemilikan_atau_setiap_perubahan_kepemilikan_saham_perusahaan_terbuka_dan_aktivitas_menjaminkan_saham_perusahaan_terbuka.pdf,21,21-25,False
2,Pembelian Kembali Saham yang Dikeluarkan oleh Perusahaan Terbuka,https://www.ojk.go.id/id/regulasi/Pages/Pembelian-Kembali-Saham-yang-Dikeluarkan-oleh-Perusahaan-Terbuka-.aspx,Pasar Modal,Peraturan Lainnya,Peraturan OJK,29 Tahun 2023,29 Desember 2023,Pembelian-Kembali-Saham-yang-Dikeluarkan-oleh-Perusahaan-Terbuka- -- POJK 29 Tahun 2023 Pembelian Kembali Saham Yang Dikeluarkan Oleh Perusahaan Terbuka.pdf,https://www.ojk.go.id/id/regulasi/Documents/Pages/Pembelian-Kembali-Saham-yang-Dikeluarkan-oleh-Perusahaan-Terbuka-/POJK%2029%20Tahun%202023%20Pembelian%20Kembali%20Saham%20Yang%20Dikeluarkan%20Oleh%20Perusahaan%20Terbuka.pdf,ojk-peraturan_ojk-29_tahun_2023-29122023-pembelian_kembali_saham_yang_dikeluarkan_oleh_perusahaan_terbuka.pdf,29,26-30,False


In [43]:
out_file = "cqa_datasets.jsonl"

# 1. Muat daftar dokumen yang sudah pernah diproses
processed = set()
if os.path.exists(out_file):
    with open(out_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                entry = json.loads(line)
                processed.add(entry["filename"])
            except Exception:
                continue

initial_count = len(processed)  # simpan berapa yang sudah ada

# 2. Open file append
interrupted = False
with open(out_file, "a", encoding="utf-8") as fout:
    try:
        for _, row in tqdm(df_metadata.iterrows(), total=len(df_metadata), desc="Dokumen"):
            fn = row["new_filename"]
            if fn in processed:
                print(f"⏭️ Skip (sudah diproses): {fn}")
                continue

            pdf_path = os.path.join(pdf_root, fn)
            if not os.path.exists(pdf_path):
                tqdm.write(f"⚠️ File tidak ditemukan: {fn}")
                processed.add(fn)
                continue

            full_text = extract_text_from_pdf(pdf_path, ocr)
            if not full_text.strip():
                processed.add(fn)
                continue

            n_pairs = get_num_pairs(int(row["pages"]))
            try:
                cqa_list = generate_cqa(full_text, n_pairs=n_pairs)
            except Exception as e:
                tqdm.write(f"⚠️ Error generate QA {fn}: {e}")
                processed.add(fn)
                continue

            if not cqa_list:
                tqdm.write(f"⏭️ Skip (tidak bermakna): {fn}")
                processed.add(fn)
                continue

            # Tulis hasil per‐dokumen langsung
            for cqa in cqa_list:
                cqa.update({
                    "file_url":           row["file_url"],
                    "regulation_number":  row["nomor_regulasi"],
                    "title":              row["title"],
                    "filename":           fn,
                    "n_pairs_requested":  n_pairs
                })
                fout.write(json.dumps(cqa, ensure_ascii=False) + "\n")
            fout.flush()
            processed.add(fn)
            tqdm.write(f"✅ {len(cqa_list)} QA untuk {fn}")

    except KeyboardInterrupt:
        interrupted = True
        print("\n⏸️ Proses dihentikan oleh pengguna. Progress sudah tersimpan.")

    finally:
        total_count = len(processed)
        added = total_count - initial_count
        if interrupted:
            print(f"🎉 Selesai sementara. {total_count} dokumen terproses total.")
        else:
            print(f"🎉 Semua dokumen selesai diproses!")
        print(f"   • Dokumen sudah ada di dataset sebelum run: {initial_count}")
        print(f"   • Dokumen baru ditambahkan di run ini : {added}")

Dokumen:   0%|          | 1/206 [00:21<1:13:30, 21.51s/it]

✅ 6 QA untuk ojk-peraturan_ojk-6_tahun_2024-02052024-pembiayaan_transaksi_efek_oleh_perusahaan_efek_bagi_nasabah_dan_transaksi_short_selling_oleh_perusahaan_efek.pdf


Dokumen:   1%|          | 2/206 [00:41<1:09:58, 20.58s/it]

✅ 5 QA untuk ojk-peraturan_ojk-4_tahun_2024-28022024-laporan_kepemilikan_atau_setiap_perubahan_kepemilikan_saham_perusahaan_terbuka_dan_aktivitas_menjaminkan_saham_perusahaan_terbuka.pdf
⏭️ Skip (sudah diproses): ojk-peraturan_ojk-29_tahun_2023-29122023-pembelian_kembali_saham_yang_dikeluarkan_oleh_perusahaan_terbuka.pdf


Dokumen:   2%|▏         | 4/206 [00:51<36:20, 10.80s/it]  

...

✅ 4 QA untuk ojk-peraturan_ojk-1_pojk_07_2014-23012014-pojk_tentang_lembaga_alternatif_penyelesaian_sengketa_di_sektor_jasa_keuangan.pdf
⏭️ Skip (sudah diproses): ojk-peraturan_ojk-5_pojk_05_2013-31122013-pojk_tentang_pengawasan_badan_penyelenggara_jaminan_sosial_oleh_otoritas_jasa_keuangan.pdf


Dokumen:  99%|█████████▊| 203/206 [24:21<00:22,  7.55s/it]

✅ 6 QA untuk ojk-peraturan_ojk-4_pojk_05_2013-23122013-pojk_tentang_penilaian_kemampuan_dan_kepatutan_bagi_pihak_utama_pada_lembaga_jasa_keuangan_non_bank.pdf
⏭️ Skip (sudah diproses): ojk-peraturan_ojk-3_pojk_05_2013-12092013-pojk_tentang_laporan_bulanan_lembaga_jasa_keuangan_non_bank.pdf
⏭️ Skip (sudah diproses): ojk-peraturan_ojk-2_pojk_04_2013-26082013-pojk_tentang_pembelian_kembali_saham_yang_dikeluarkan_oleh_emiten_atau_perusahaan_publik_dalam_kondisi_pasar_yang_berfluktuasi_secara_signifikan.pdf


Dokumen: 100%|██████████| 206/206 [24:41<00:00,  7.19s/it]

✅ 7 QA untuk ojk-peraturan_ojk-1_pojk_07_2013-06082013-pojk_tentang_perlindungan_konsumen_sektor_jasa_keuangan.pdf
🎉 Semua dokumen selesai diproses!
   • Dokumen sudah ada di dataset sebelum run: 100
   • Dokumen baru ditambahkan di run ini : 106


### **Check Dataset**

In [46]:
# Baca seluruh JSONL ke DataFrame
df_results = pd.read_json("cqa_datasets.jsonl", lines=True)

# Cek ukuran dan beberapa baris pertama
print("Total QA pairs:", len(df_results))
df_results[["context", "question", "answer"]].sample(5)

Total QA pairs: 868


,context,question,answer
625,"Peraturan Otoritas Jasa Keuangan Republik Indonesia Nomor 35/POJK.03/2019 mengatur tentang transparansi kondisi keuangan Bank Pembiayaan Rakyat Syariah (BPRS). Tujuan dari peraturan ini adalah untuk meningkatkan transparansi dengan mengharuskan BPRS untuk mengumumkan laporan keuangan secara lengkap, akurat, dan dapat diperbandingkan. Laporan ini harus disusun sesuai dengan standar akuntansi keuangan yang berlaku dan disampaikan kepada Otoritas Jasa Keuangan dalam waktu yang telah ditetapkan.",Apa tujuan dari Peraturan Otoritas Jasa Keuangan Nomor 35/POJK.03/2019?,Tujuan dari peraturan ini adalah untuk meningkatkan transparansi kondisi keuangan BPRS dengan mewajibkan mereka mengumumkan laporan keuangan secara lengkap dan akurat.
616,"Peraturan Otoritas Jasa Keuangan Republik Indonesia Nomor 2/POJK.04/2020 mengatur tentang perubahan atas Peraturan Otoritas Jasa Keuangan Nomor 23/POJK.04/2016 mengenai Reksa Dana Berbentuk Kontrak Investasi Kolektif. Dalam peraturan ini, terdapat penekanan pada perlindungan investor Reksa Dana dan profesionalisme Manajer Investasi. Pengaturan ini juga mempertimbangkan manajemen risiko diversifikasi Efek dalam portofolio investasi Reksa Dana yang sebelumnya dinilai tidak sesuai dengan upaya perlindungan investor.",Apa tujuan dari Peraturan Otoritas Jasa Keuangan Nomor 2/POJK.04/2020?,Tujuan dari Peraturan ini adalah untuk meningkatkan perlindungan investor Reksa Dana dan profesionalisme Manajer Investasi dalam pengelolaan Reksa Dana serta memperbaiki manajemen risiko diversifikasi Efek.
204,"Peraturan Otoritas Jasa Keuangan Republik Indonesia Nomor 14/POJK.04/2022 bertujuan untuk mengatur penyampaian laporan keuangan berkala oleh emiten atau perusahaan publik. Dalam pertimbangan, diungkapkan bahwa perlu ada penyesuaian terhadap regulasi yang ada untuk meningkatkan efisiensi dan memenuhi kebutuhan pasar saat ini. Perubahan ini juga untuk mencerminkan praktik terbaik yang berlaku di negara lain, agar laporan keuangan dapat diakses dengan cepat oleh publik, terutama pemegang saham.",Apa tujuan dari Peraturan OJK Nomor 14/POJK.04/2022?,Tujuan dari Peraturan OJK Nomor 14/POJK.04/2022 adalah untuk mengatur penyampaian laporan keuangan berkala emiten atau perusahaan publik agar lebih efisien dan sesuai dengan perkembangan pasar saat ini.
402,"Tim Likuidasi memiliki peran penting dalam proses likuidasi dana pensiun sesuai dengan ketentuan dalam Peraturan OJK. Tim ini bertugas menyusun rencana kerja, melakukan inventarisasi kekayaan dan kewajiban, serta menyampaikan informasi kepada peserta dan pensiunan. Anggota tim harus memiliki kompetensi yang sesuai dan tidak memiliki catatan pelanggaran dalam sektor jasa keuangan. Tanggung jawab Tim Likuidasi juga meliputi menyelesaikan kewajiban keuangan dana pensiun dan melaporkan perkembangan likuidasi kepada OJK.",Apa saja tugas Tim Likuidasi dalam proses likuidasi dana pensiun?,"Tugas Tim Likuidasi antara lain menyusun rencana kerja, melakukan inventarisasi kekayaan, menyampaikan informasi kepada peserta, dan menyelesaikan kewajiban keuangan dana pensiun serta melaporkan kepada OJK."
415,"Peraturan ini juga mengatur sanksi bagi BPRS yang tidak memenuhi kewajiban modal. Sanksi administratif dapat berupa teguran tertulis, penurunan tingkat kesehatan, larangan pembukaan jaringan kantor, hingga penghentian sementara sebagian kegiatan operasional. Hal ini bertujuan untuk mendorong kepatuhan terhadap ketentuan yang berlaku.",Apa sanksi yang dapat dikenakan pada BPRS yang tidak memenuhi kewajiban modal?,"Sanksi yang dapat dikenakan pada BPRS yang tidak memenuhi kewajiban modal meliputi teguran tertulis, penurunan tingkat kesehatan, dan larangan pembukaan jaringan kantor."
